In [40]:
import os
import torch
from torch import nn                            # 신경망모듈
from torch.utils.data import DataLoader         # 데이터 로딩 관련 모듈
from torchvision import datasets, transforms    # torch저장소 데이터 셋 모듈

#### [1] 데이터 불러오기

In [41]:
# Tensor의 저장소 설정을 위한 체크
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


#### [2] 신경망 모델 생성

In [42]:
# 순차적인 Layer 구성 => Sequential, Layer...
class NeuralNetwork(nn.Module):

    # 생성자 메서드, NeuralNetwork() 객체 생성 시 실행
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # 입력층
            nn.Linear(28*28, 512),
            nn.ReLU(),

            # 은닉층
            nn.Linear(512, 512),
            nn.ReLU(),

            # 출력층
            nn.Linear(512, 10),
        )
        
    # NN 모델이 동작을 시작할 떄 실행
    def forward(self, x):
        # 1차원으로 데이터 변환
        x = self.flatten(x)
        # NN 모델에 입력 데이터 전달
        logits = self.linear_relu_stack(x)
        return logits

In [43]:
# NN 모델 생성
mnistM = NeuralNetwork().to(device)

In [44]:
print(mnistM)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [45]:
# 데이터 준비
X = torch.rand(1, 28, 28, device=device)

In [46]:
inptT=torch.rand(1, 28, 28)

In [47]:
print(X)

tensor([[[3.7725e-01, 1.5910e-01, 9.6034e-02, 8.6071e-01, 1.0118e-01,
          1.8592e-01, 5.2255e-01, 3.9052e-01, 7.7449e-01, 2.0657e-01,
          2.2475e-01, 4.0996e-01, 1.2159e-01, 3.9696e-01, 6.3821e-01,
          2.9391e-01, 6.3074e-01, 9.9941e-01, 5.0215e-01, 7.4630e-01,
          1.6576e-01, 5.1425e-02, 7.4109e-01, 6.1974e-01, 1.7814e-01,
          6.1162e-01, 6.6012e-02, 6.9078e-01],
         [9.3643e-01, 5.1638e-01, 4.0024e-01, 3.8758e-01, 1.7174e-01,
          1.6643e-01, 7.7668e-02, 6.1221e-02, 4.6311e-01, 6.3531e-01,
          8.3729e-01, 6.9308e-01, 4.1471e-01, 2.0229e-01, 3.2776e-01,
          7.1187e-01, 3.2500e-01, 2.3862e-01, 3.3156e-01, 2.0628e-01,
          1.6535e-01, 9.5190e-01, 5.6204e-01, 9.0037e-01, 2.2389e-01,
          6.9918e-01, 4.4861e-01, 7.5846e-01],
         [2.9882e-02, 2.1264e-01, 7.5797e-01, 8.4103e-02, 9.9271e-01,
          9.0550e-01, 6.6602e-01, 6.0551e-01, 5.7866e-01, 3.7327e-01,
          4.8497e-01, 3.4007e-01, 5.2773e-01, 3.9373e-01, 9.7249e-

In [48]:
# 학습
logits = mnistM(X)

In [49]:
logits

tensor([[-0.1540,  0.0214, -0.1131, -0.0826, -0.0724, -0.0207,  0.0474,  0.0510,
         -0.0006, -0.0607]], grad_fn=<AddmmBackward0>)

In [50]:
# 결과 분석
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class : {y_pred}')

Predicted class : tensor([7])


In [51]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")